In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Вспомогательные функции

In [ ]:
def hist(df, columns, nrows, ncols, figsize=(600, 1500)):
    fig = make_subplots(rows=nrows, cols=ncols)
    for idx, column in enumerate(columns):
        fig.append_trace(go.Histogram(x=df[column], texttemplate="%{x}", name=column), 
                        idx//ncols + 1, idx%ncols + 1)
    fig.update_layout(
        autosize=False,
        height=figsize[0],
        width=figsize[1],
        margin=dict(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4
        ),
        # paper_bgcolor="LightSteelBlue",
    )
    fig.show()

In [ ]:
def ind_hists(df, columns, figsize=(400,1500), marginal=None, nbins=None, log=(True, True)):
    for idx, column in enumerate(columns):
        px.histogram(df, column, height=figsize[0], width=figsize[1], marginal=marginal, nbins=nbins, log_x=log[0], log_y=log[1]).show()

In [ ]:
def del_right_outliers(df, column, q):
    q = df[column].quantile(q)
    print(f"Удаление значений, больше {q}. Это {df[df[column] >= q].shape[0]} записей")
    return df[df[column] < q]

In [ ]:
def split_by_zeros(df, columns, threshold):
    idx = (df[columns.keys()] == 0).sum(axis=1) >= threshold
    return df.loc[idx, :], df.loc[~idx, :]

In [ ]:
def value_counts(df, columns):
    for col in columns:
        print(df[col].value_counts(), "\n", "+++", "="*len(col), "+++")

# EDA

## Первичный анализ

- Парсинг временны'х признаков
- Добавление смещения значений по добыче воды за 2 часа
- Фильтрация времени, оставляя ручные измерения (8 утра/вечера)

In [ ]:
df_original.head(3)

- shape=(4334, 59)
- Columns=['Год', 'Месяц', 'День', 'Час', 'Блок манифольда|P бм  |кгс/см²',
       'Блок манифольда|t жидкости| °С', 'С-1|P сеп. |кгс/см²',
       'С-1|t жидкости |°С', 'С-1|L жидкости| см', 'С-2/1|P сеп. |кгс/см²',
       'С-2/1|t жидкости|°С', 'С-2/1|L межфазный|см', 'С-2/1|L нефти|см',
       'С-2/2|P сеп. |кгс/см²', 'С-2/2|t жидкости|°С', 'С-2/2|L межфазный|см',
       'С-2/2|L нефти|см', 'ОН-1/1|P отс. |кгс/см²', 'ОН-1/1|t жидкости|°С',
       'ОН-1/1|L межфазный|см', 'ОН-1/2|P отс. |кгс/см²',
       'ОН-1/2|t жидкости|°С', 'ОН-1/2|L  межфазный|см', 'С-3|t жидкости|°С',
       'С-3|L нефти|см', 'П-1|Т нефти на входе|°С', 'П-1|Т нефти на выходе|°С',
       'П-1|Р нефти на входе|кгс/см²', 'П-1|Р нефти на выходе|кгс/см²',
       'П-1|Т теплоно-сителя|°С', 'П-1|Т дымовых газов|°С',
       'П-1|Р на горелку|кгс/см2', 'П-2|Т нефти на входе|°С',
       'П-2|Т нефти на выходе|°С', 'П-2|Р нефти на входе|кгс/см²',
       'П-2|Р нефти на выходе|кгс/см²', 'П-2|Т теплоно-сителя|°С',
       'П-2|Т дымовых газов|°С', 'П-2|Р на горелку|кгс/см3',
       'П-3|Т нефти на входе|°С', 'П-3|Т нефти на выходе|°С',
       'П-3|Р нефти на входе|кгс/см²', 'П-3|Р нефти на выходе|кгс/см²',
       'П-3|Т теплоно-сителя|°С', 'П-3|Т дымовых газов|°С',
       'П-3|Р на горелку|кгс/см4', 'БЕВ-1|L воды|см', 'БЕВ-1|V  воды|м3',
       'БЕВ-1|V нефти|м3', 'БЕВ-1|t воды|°С', 'БЕВ-2|L воды|см',
       'БЕВ-2|V  воды|м3', 'БЕВ-2|V нефти|м3', 'БЕВ-2|t воды|°С',
       'БЕВ-3|L воды|см', 'БЕВ-3|V  воды|м3', 'БЕВ-3|V нефти|м3',
       'БЕВ-3|t воды|°С', 'Вода с лагом (-1)']
- float64(22), int32(4), int64(33)
- Null значений нету

## Анализ нулевых значений

In [ ]:
zero_columns = {column: (df[column]==0).sum()/df[column].count() * 100 
                for column 
                in df.columns
                if (df[column]==0).sum()!=0}

zero_columns = dict(sorted(zero_columns.items(), 
                           key=lambda item: item[1], 
                           reverse=True))
print(*[f'{idx:2}.  {zero_percent:5.2f}: {column}' for idx, (column, zero_percent) in enumerate(zero_columns.items())], 
        sep='\n')

Уберем из списка нулевых столбцов те, у которых меньше 1% нулей

In [ ]:
zero_columns = slice(zero_columns, stop=20)


Alarm:

 0.  100.00: БЕВ-2|V нефти|м3
 1.  100.00: БЕВ-3|V нефти|м3
 2.  95.34: БЕВ-1|V нефти|м3
 3.  80.94: БЕВ-1|t воды|°С
 4.  80.94: БЕВ-2|t воды|°С
 5.  80.94: БЕВ-3|t воды|°С
 6.  44.23: П-1|Т нефти на входе|°С
 7.  44.23: П-1|Т нефти на выходе|°С
 8.  44.23: П-1|Р нефти на входе|кгс/см²
 9.  44.23: П-1|Р нефти на выходе|кгс/см²
10.  44.23: П-1|Т теплоно-сителя|°С
11.  44.23: П-1|Т дымовых газов|°С
12.  44.23: П-1|Р на горелку|кгс/см2

In [ ]:
bevs = list(slice(zero_columns, stop=6).keys())
hist(df, bevs, 3, 2, figsize=(800,1500))

- В БЕВ'ав слишком много нулевых значений => их больше не учитываем

In [ ]:
df.drop(columns=bevs, inplace=True)
zero_columns = slice(zero_columns, start=6)
zero_columns

In [ ]:
p_1 = list(slice(zero_columns, stop=7).keys())
ind_hists(df, p_1, marginal='box', nbins=200, log=(False,True))

> Выбросы в: 
> - "П-1|Р нефти на выходе|кгс/см²" (43.1, count=1)
> - "П-1|Р на горелку|кгс/см2': 44.23165666820489" (300, count=1)

> Удаляем...

In [ ]:
df = del_right_outliers(df, 'П-1|Р нефти на выходе|кгс/см²', .999)
df = del_right_outliers(df, 'П-1|Р на горелку|кгс/см2', .9991)


> Снова смотрим на распределения

In [ ]:
ind_hists(df, p_1, marginal='box', nbins=200, log=(False,True))

> Стало немного лучше

In [ ]:
df.shape, df_original.shape

> Однако, в первой печке тоже много значений по нулям
- Посмотрим, сколько строк имеют большое кол-во нулей

In [ ]:
threshold = 7
df_much_zeros, df_without_zeros = split_by_zeros(df, zero_columns, threshold)
print(f"Датасет содержащий строки с кол-во нулей в столбцах < {threshold}: shape=({df_without_zeros.shape})")
print(f"Датасет содержащий строки с кол-вом нулей в столбцах >= {threshold}: shape=({df_much_zeros.shape})")

## Обработка выбросов

In [2]:
outlier_columns = [col for col in df if 'outlier' in col]
outlier_columns.append('Вода с лагом (-1)')

orig_columns = [col for col in df if col not in outlier_columns]
orig_columns.append('Добыча воды за 2 ч |м3 лаг:(-1)')
orig_columns.remove('YY-MM-DD HH:00')

pechki_columns = [col for col in df if 'П' in col]
pechki_columns.append('Добыча воды за 2 ч |м3 лаг:(-1)')


### Scatter matrixes

In [ ]:
for i in range(0, len(outlier_columns), 10):
    df.scatter_matrix(outlier_columns[i: i+11], join(PLOTS, f'scatter_matrix{i}.html'), size=(4000, 4000))

In [3]:
df.scatter_matrix(orig_columns, join(PLOTS, 'all_scatter_matrix.html'), size=(4000, 4000))
df.scatter_matrix(pechki_columns, join(PLOTS, 'печки_scatter_matrix.html'), size=(4000, 4000))

### Correlation matrixes

In [4]:
df.corr_matrix(pechki_columns, filepath=join(PLOTS, 'печки_corr_matrix.html'),
               size=(4000, 4000))

##  Проверка колебаний аналогичных показателей во времени

In [5]:
df.time_series(columns='П', appendix_cols=['Добыча воды за 2 ч |м3 лаг:(-1)'], log=(False,False),
               save_path=join(PLOTS, 'Печки.html'), show=False)

df.time_series(columns='С-', appendix_cols=['Добыча воды за 2 ч |м3 лаг:(-1)'], log=(False,False),
               save_path=join(PLOTS, 'Сепараторы.html'), show=False)

df.time_series(columns='ОН', appendix_cols=['Добыча воды за 2 ч |м3 лаг:(-1)'], log=(False,False),
               save_path=join(PLOTS, 'ОН.html'), show=False)